In [60]:
import asyncio
from twikit import Client, TooManyRequests
from configparser import ConfigParser
import pandas as pd
from datetime import datetime, timedelta

# Constantes
MINIMUMS = 200  
REQUETE = 'iphone 15 | nouvel-iphone -Fr'  
# Lire les informations de connexion depuis le fichier config.ini
config = ConfigParser()
config.read('config.ini')
username = config['X']['username']
email = config['X']['email']
password = config['X']['password']

client = Client(language='fr-FR')  # Configurer pour obtenir des résultats en français

# Fonction principale async
async def main():
    # Authentification
    await client.login(auth_info_1=username, auth_info_2=email, password=password)
    
    # Sauvegarder les cookies après l'authentification
    client.save_cookies('cookies.json')
    client.load_cookies('cookies.json')  # Charger les cookies pour les sessions futures

    # Initialisation de la liste pour les tweets et compteur
    donne = []
    tweet_count = 0

    # Rechercher les tweets avec la requête spécifiée
    tweets = await client.search_tweet(REQUETE, product='Top')

    # Boucle pour paginer les tweets
    while tweet_count < MINIMUMS:
            # Récupérer la page suivante de tweets
            encore = await tweets.next()

            for tweet in encore:
                post_data = {
                    'Texte': tweet.full_text,
                    'Nombre_Likes': tweet.favorite_count,
                    'Nombre_Retweets': tweet.retweet_count,
                    'Followers_Utilisateur': tweet.user.followers_count,
                    'Utilisateur': tweet.user.screen_name,
                    'Date_Publication': tweet.created_at
                }
                donne.append(post_data)
                tweet_count += 1

                # Vérifier si nous avons atteint le nombre minimum de tweets
                if tweet_count >= MINIMUMS:
                    break
            # Sortir de la boucle si aucun tweet supplémentaire n'est disponible
        

    # Vérifier que des tweets ont été collectés
    if donne:
        df = pd.DataFrame(donne)
        print(df)
    else:
        print("Aucune donnée collectée.")

# Exécuter la fonction async
await main()


                                                 Texte  Nombre_Likes  \
0    J'ai acheté l'iPhone 15 en Bleu à seulement 70...           314   
1       iPhone 15 est sorti que dieu protège nos Soeur           481   
2    Les #iPhone15 arrivent (déjà) vendredi prochai...           569   
3    📱🍏 Prix des nouveaux iPhone 15 :\n\n- iPhone 1...          2376   
4    🔴 Finalement, j’ai été très sage cette année. ...           331   
..                                                 ...           ...   
195  Quelques retours sur le fameux #iPhone15ProMax...           202   
196  🍎 FLASH - iOS 15 est officiellement disponible...          7253   
197  🔥 Nouvelle vidéo en ligne !\n\nN’achetez pas l...           243   
198  📲 FLASH - Les prix des nouveaux iPhone sont lé...         12324   
199  🚀 \nC'est parti pour les préco iPhone 15 \nRéc...           166   

     Nombre_Retweets  Followers_Utilisateur      Utilisateur  \
0                 17                   2076        nakryumyt   
1      